Data Collocation

In [ ]:
# taamolat data set
taamolat_directory  = 'Authors Dataset/taamolat/'
TAAMOLAT_NUMBER_OF_AUTHORS = 7
TAAMOLAT_NUMBER_OF_SAMPLES_PER_AUTHOR = 10

# alaraby data set
alaraby_directory = 'Authors Dataset/alaraby/'
ALARABY_NUMBER_OF_AUTHORS = 9
ALARABY_NUMBER_OF_SAMPLES_PER_AUTHOR = 10

Function for return the list of authors names for a specific dataset

In [79]:
def get_authors_names(directory_name, number_of_authors):
    import configparser

    # Create a ConfigParser object
    config = configparser.ConfigParser()

    # save the authors names in a list
    authors = []
    for i in range(number_of_authors):
        # Read the properties file with the correct encoding (e.g., 'utf-8')
        with open(f'{directory_name}a{i}/a{i}_0.properties', encoding='utf-8') as f:
            # Add a default section header to the file
            content = '[default]\n' + f.read()
            config.read_string(content)

        # Get the author name
        # Get the value of the 'author_name' key
        author_name = config.get('default', 'author_name')
        authors.append(author_name)

    return authors

In [80]:
taamolat_authors_english = get_authors_names(taamolat_directory, TAAMOLAT_NUMBER_OF_AUTHORS)
# manually add the arabic names of the authors of the taamolat dataset
taamolat_authors_names = ['محمد عواد', 'محمد زينوبة', 'يونس جعدي', 'نور الدين رافع', 'عبير قلوطي', 'معتز خطيب', 'إبراهيم نصر الله']
taamolat_authors = [ i for i in range(TAAMOLAT_NUMBER_OF_AUTHORS) ]
alaraby_authors_names = get_authors_names(alaraby_directory, ALARABY_NUMBER_OF_AUTHORS)
alaraby_authors = [ i for i in range(ALARABY_NUMBER_OF_AUTHORS) ]

print(taamolat_authors[0], taamolat_authors_names[0])
print(alaraby_authors[0], alaraby_authors_names[0])


0 محمد عواد
0 عبد الله الشريف


Read the paragraphs of the authors 

In [81]:
# Get each paragraph with its author
def get_paragraphs_with_authors(directory_name, number_of_authors, number_of_samples_per_author, authors_names):
    paragraphs = []
    for i in range(number_of_authors):
        for j in range(number_of_samples_per_author):
            with open(f'{directory_name}a{i}/sample{j}.txt', encoding='utf-8') as file:
                current_paragraph = ''
    
                for line in file:
                    line = line.strip()
                
                    if line:  # If the line is not empty
                        current_paragraph += line + ' '
                    elif current_paragraph:  # If an empty line is encountered and there is a current paragraph
                        paragraphs.append((current_paragraph.strip(), authors_names[i]))
                        current_paragraph = ''

                if current_paragraph:  # Append the last paragraph if it exists after reaching the end of the file
                    paragraphs.append((current_paragraph.strip(), authors_names[i]))
    return paragraphs

In [82]:
taamolat_data = get_paragraphs_with_authors(taamolat_directory, TAAMOLAT_NUMBER_OF_AUTHORS, TAAMOLAT_NUMBER_OF_SAMPLES_PER_AUTHOR, taamolat_authors)
alaraby_data = get_paragraphs_with_authors(alaraby_directory, ALARABY_NUMBER_OF_AUTHORS, ALARABY_NUMBER_OF_SAMPLES_PER_AUTHOR, alaraby_authors)

# for paragraph in taamolat_data:
#     print(paragraph)
#     print()

# for paragraph in alaraby_data:
#     print(paragraph)
#     print()


# combine the two datasets
data_set = taamolat_data + alaraby_data
# make sure that the data is shuffled
import random
random.shuffle(data_set)

number_of_samples = len(data_set)


for paragraph in data_set:
    print(paragraph)
    print()

('أما المقاصد التابعة:', 5)

('قد تكون التركيبة البشرية للجيش في عمقه الصلب قد تغيّرت بفعل جيل عسكري شاب يستند إلى التقنية وبمستويات علمية وإلى شرعية الكفاءة والأداء، بخلاف الزمن السابق الذي كانت فيه الشرعية الثورية المستند الأبرز. وقد تكون الظروف المحلية تغيّرت على أكثر من صعيد في علاقة بالبنية السياسية والمجتمعية، لكن ما لم يتغيّر في الواقع هو علاقة الجيش بالحكم، والتي ظلت دائماً تحتكم إلى القاعدة نفسها، إذ يمسك الجيش بالسلطة في الخلف ويقبع الحاكم المدني في الواجهة السياسية.', 6)

('رغم كلّ مساوئه', 6)

('ولعل أجمل ما شهده حفل نهايات الأسبوع الماضي، أن الذين يعرفون الأغاني لم ينسوها، ولم يملّوا سماعها، والذين سمعوها للمرة الأولى، أحسوها بأنها أغنيات جديدة تولد الآن. لقد اختُبِرَت أغاني بلدنا أربعين عامًا بالزمن، وأثبتت أنها قادرة على أن تظلّ على قيد الحياة، شابة في مقتبل عمرها، وهي في الأربعين.', 6)

('وكذلك المدينة تتكون من ثلاث طبقات: الرؤساء والجنود والعمال والفلاحين. وهذه الطبقات الثلاث لنفس الفرد والمدينة تدور على ثلاث فضائل: فالنفس الغضبية تدور على فضيلة الشجاعة، والنفس الشهوية

Preprocessing The data sets

In [83]:

# Normalization: remove diacritics from the text & Punctuation mark removal & Numbers & Non-arabic & alef, teh 
def normalization(text):
    import re
    import tnkeeh
    from pyarabic.araby import strip_tashkeel, normalize_alef, normalize_teh
    text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    text = strip_tashkeel(text)
    text = normalize_alef(text)
    text = normalize_teh(text)
    tnkeeh_obj = tnkeeh.Tnkeeh(
    remove_special_chars=True,
    remove_english=True, normalize=True, remove_diacritics=True,
    remove_tatweel=True, remove_html_elements=True,
    remove_links=True, remove_twitter_meta=True
    )
    text = tnkeeh_obj.clean_raw_text(text)[0]
    
    return text

text = "يْشَارْ إِلَasfgى ةهأغإلأألأَنْ اللُّغَ15+62ة الْعَ?!#@$#^رْبِيَّة"
text = normalization(text)
print(text)





يشار الا ههاغالاالان اللغه العربيه


In [84]:

# stemming
def stemming(word):
    from nltk.stem.isri import ISRIStemmer # special stemmer for arabic
    # Create an instance of the ISRI Arabic Stemmer

    stemmer = ISRIStemmer()
    return stemmer.stem(word)

text = 'المدرسة'
result = stemming(text)
print(result)



درس


In [85]:
# POS tagging
def pos_tagging(text):

    # from farasa.pos import FarasaPOSTagger
    # tagger = FarasaPOSTagger()
    # tags = tagger.tag_segments(text, combine_subtokens=True)
    # return tags

    # import json
    # import requests
    # api_key = "mgmfGsWLmIFJznbMaC"

    # url = 'https://farasa.qcri.org/webapi/pos/'
    # payload = {'text': text, 'api_key': api_key}
    # data = requests.post(url, data=payload)
    # result = json.loads(data.text)

    # result = result['text']

    # words_pos_tags = []
    # for obj in result:
    #     words_pos_tags.append((obj['surface'], obj['POS']))

    # # remove the first and last elements of the list
    # words_pos_tags = words_pos_tags[1:-1]

    # return words_pos_tags
    pass


text = 'يُشار إلى أن اللغة العربية'
result = pos_tagging(text)
print(result)






None


In [86]:
# tokenization
def tokenization(text):
    from pyarabic.araby import tokenize
    
    text = tokenize(text)
    return text

text = 'يُشار إلى أن اللغة العربية'

result = tokenization(text)
print(result)



['يُشار', 'إلى', 'أن', 'اللغة', 'العربية']


Arabic Stop Words

In [87]:
from nltk.corpus import stopwords

stop_words_file_name = 'stop_words.txt'
stop_words = []
with open(stop_words_file_name, encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        stop_words.append(line)

stop_words = list(set(stop_words))
stop_words += stopwords.words('arabic')

print(stop_words)

['', 'اللاتي', 'أهلا', 'بين', 'مع', 'وقد', 'شتانَ', 'وقالت', 'عندما', 'إليكَ', 'حجا', 'صراحة', 'ياء', 'أطعم', 'إليكنّ', 'سوى', 'اربعة', 'كأن', 'أولاء', 'وإن', 'يناير', 'وكل', 'سوف', 'جيم', 'إذاً', 'دواليك', 'صبرا', 'ظ', 'سنة', 'ست', 'عوض', 'غ', 'ح', 'ستمائة', 'صاد', 'سنوات', 'سبعمئة', 'اليه', 'هَؤلاء', 'اضحى', 'ذ', 'إياكما', 'اكد', 'لكنَّ', 'أضحى', 'أنه', 'أعلم', 'تانِك', 'وقال', 'كيف', 'أربعة', 'خلال', 'فان', 'ما', 'عاد', 'مليون', 'نا', 'إياه', 'ستكون', 'عدد', 'أبدا', 'ذا', 'رأى', 'تسعون', 'جميع', 'معاذ', 'بئس', 'وأبو', 'سقى', 'دولار', 'طاء', 'نفسها', 'أيضا', 'كن', 'مليم', 'انبرى', 'بسّ', 'امس', 'ى', 'كلتا', 'جدا', 'فلس', 'واكد', 'اطار', 'والتي', 'حيثما', 'تينك', 'منذ', 'نهاية', 'شبه', 'التى', 'ألف', 'ستة', 'ص', 'عيانا', 'لبيك', 'تِه', 'مه', 'هما', 'كاف', 'هنا', 'عنه', 'خاصة', 'كل', 'لديه', 'ومع', 'الذاتي', 'فما', 'برس', 'تلقاء', 'لات', 'ما أفعله', 'علًّ', 'مما', 'بلى', 'سبعون', 'م', 'الحالي', 'أخٌ', 'أمّا', 'أنشأ', 'ديك', 'فعل', 'مهما', 'هى', 'راح', 'الذى', 'ثمَّ', 'تموز', 'كما', 'مر

In [88]:
def remove_stop_words(words, stop_words=stop_words):
    
    result = [] 
    removed_indexes = []
    for i in range(len(words)):
        if words[i] in stop_words:
            removed_indexes.append(i)
        else:
            result.append(words[i])
        
    removed_indexes.sort(reverse=True) # for save remove of pos tagging 
    return result, removed_indexes

remove_stop_words(['هذا', 'هو', 'من', 'الكلمات', 'المستبعدة', 'من', 'التحليل'])

(['الكلمات', 'المستبعدة', 'التحليل'], [5, 2, 1, 0])

Preprocessing: at the beginning we made normlization for the text, then make tokenization for remove the stop words and apply stemming on the words without stop words, 
after that find the pos tagging on the normalized text with stop words to make corrcot results, the remove the pos tags of the stop words, and just save the stemming words with thier pos tagging

In [89]:

# def preprocessing(text):
#     normalization_text = normalization(text)
#     segments = tokenization(normalization_text)

#     # remove stop words from the the text after normalization
#     words, removed_indexes = remove_stop_words(segments)
#     number_of_words = len(words)
    
#     # find the pos tags for the words in the original text after normalization
#     tags = pos_tagging(normalization_text)
   

#     # remove the tags of the stop words
#     for index in removed_indexes:
#         tags.pop(index)

#     # stemming for the words in the original text after tokenization without stop words
#     for i in range(number_of_words):
#         words[i] = stemming(words[i])

#     sample = []

#     # combine the stemming words with their pos tags
#     for i in range(number_of_words):
#         sample.append((words[i], tags[i][1]))

    
#     return sample


def preprocessing(text):
    normalization_text = normalization(text)
    segments = tokenization(normalization_text)

    # remove stop words from the the text after tokenization
    words, _ = remove_stop_words(segments)
    number_of_words = len(words)
    
    
    # stemming for the words in the original text after tokenization without stop words
    for i in range(number_of_words):
        words[i] = stemming(words[i])

 
    return words




text = "أهلاً بكم! هذه الجملة تحتاج للتنظيف. 123"

result = preprocessing(text)
print(result)
for word in result:
    print(word)

['اهل', 'جمل', 'حاج', 'نظف']
اهل
جمل
حاج
نظف


Preprocessing our dataset

In [90]:
for i in range(number_of_samples):
    data_set[i] = (preprocessing(data_set[i][0]), data_set[i][1]) # (words, author)

# print(data_set)
# for sample in data_set:
#     print(sample)
#     print()

Features Extraction

In [91]:
# TF-IDF
def tf_idf(corpus):
    from sklearn.feature_extraction.text import TfidfVectorizer


    # Extract the sentences from the corpus
    sentences = [sample[0] for sample in corpus]

    # Create a list of authors
    authors = [sample[1] for sample in corpus]

    # Convert the sentences to strings
    sentences = [' '.join(sentence) for sentence in sentences]

    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the sentences
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Get the feature names (unique words) as columns
    feature_names = vectorizer.get_feature_names()

    # Print the feature names
    print("Feature Names:", feature_names)
    print("Number of Features:", len(feature_names))

    # Print the TF-IDF matrix
    print("TF-IDF Matrix:")
    print(tfidf_matrix.toarray())
    
    # Print the first row of the TF-IDF matrix
    print(tuple(tfidf_matrix.toarray()[0]))

    # Print the author names
    print("Authors:", authors)

    return tfidf_matrix.toarray(), feature_names, authors

        
print(f'len: {len(data_set)}')
tf_idf(data_set)

len: 1594
Feature Names: ['ؤثر', 'ؤجج', 'ؤجل', 'ؤخر', 'ؤده', 'ؤرق', 'ؤسء', 'ؤسس', 'ؤسف', 'ؤشر', 'ؤقت', 'ؤكد', 'ؤكل', 'ؤلف', 'ؤلم', 'ؤمر', 'ؤمن', 'ؤنث', 'ؤنس', 'ؤهل', 'ؤون', 'ؤيد', 'ئات', 'ئتم', 'ئلف', 'ئما', 'ئوه', 'ائت', 'ائتلافيه', 'ائتمانيه', 'ائم', 'ائه', 'االتاريخ', 'اب', 'ابا', 'ابارتهايد', 'ابب', 'ابت', 'ابتسامته', 'ابح', 'ابد', 'ابر', 'ابريلنيس', 'ابض', 'ابع', 'ابق', 'ابك', 'ابل', 'ابن', 'ابه', 'ابو', 'ابي', 'ات', 'اتا', 'اتالانتا', 'اتت', 'اتج', 'اتح', 'اتر', 'اتف', 'اتق', 'اتل', 'اتم', 'اتن', 'اتو', 'اتي', 'اثب', 'اثخ', 'اثر', 'اثل', 'اثم', 'اثن', 'اثنياتالملل', 'اثو', 'اجئ', 'اجب', 'اجتماعات', 'اجتماعيه', 'اجتماعيهالقاعده', 'اجتهاديه', 'اجج', 'اجد', 'اجدادكالمتجذره', 'اجر', 'اجل', 'اجه', 'احا', 'احب', 'احت', 'احتجاجيه', 'احتكاريه', 'احتياجات', 'احتياطيه', 'احج', 'احد', 'احذ', 'احز', 'احس', 'احص', 'احض', 'احط', 'احق', 'احك', 'احل', 'احم', 'احه', 'احي', 'اخا', 'اخت', 'اختباراته', 'اختصاصيه', 'اختلافات', 'اختياراته', 'اخد', 'اخذ', 'اخر', 'اخراللاجئ', 'اخرالمجتمع', 'اخرالموت', '

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.10086122, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 ['ؤثر',
  'ؤجج',
  'ؤجل',
  'ؤخر',
  'ؤده',
  'ؤرق',
  'ؤسء',
  'ؤسس',
  'ؤسف',
  'ؤشر',
  'ؤقت',
  'ؤكد',
  'ؤكل',
  'ؤلف',
  'ؤلم',
  'ؤمر',
  'ؤمن',
  'ؤنث',
  'ؤنس',
  'ؤهل',
  'ؤون',
  'ؤيد',
  'ئات',
  'ئتم',
  'ئلف',
  'ئما',
  'ئوه',
  'ائت',
  'ائتلافيه',
  'ائتمانيه',
  'ائم',
  'ائه',
  'االتاريخ',
  'اب',
  'ابا',
  'ابارتهايد',
  'ابب',
  'ابت',
  'ابتسامته',
  'ابح',
  'ابد',
  'ابر',
  'ابر

In [ ]:
# Word Embedding
def word_embedding(corpus):
    from gensim.models import FastText
    import numpy as np

    # Extract the sentences from the corpus
    sentences = [sample[0] for sample in corpus]

    # Create a list of authors
    authors = [sample[1] for sample in corpus]

    # Train FastText model
    model = FastText(sentences, vector_size=100, window=5, min_count=2, workers=4, sg=0)


    # Get the feature names (labeled dimensions) for FastText
    feature_names = [f't{i}' for i in range(model.vector_size)]

    # Create feature vectors for each sentence
    feature_vectors = []
    for sentence in sentences:
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        if vectors:
            sentence_vector = np.mean(vectors, axis=0)
        else:
            sentence_vector = np.zeros(model.vector_size)
        feature_vectors.append(sentence_vector)

    # Convert feature vectors to NumPy array
    feature_vectors = np.array(feature_vectors)

    # Print the feature vectors
    print("Feature Vectors:")
    print(feature_vectors)

    # Print the first feature vector
    print("First Feature Vector:", feature_vectors[0].tolist())

    # Print the author names
    print("Authors:", authors)

    # Print the feature names
    print("Feature Names:", feature_names)

    return feature_vectors, feature_names, authors

word_embedding(data_set)


Combine features vectors (TF-IDF + FastText)

In [93]:
tf_idf_matrix, tf_idf_feature_names, authors_list = tf_idf(data_set)
word_embedding_matrix, word_embedding_feature_names, authors_list = word_embedding(data_set)


# Combine the feature names
combined_feature_names = tf_idf_feature_names + word_embedding_feature_names

def combine_feature_vectors(x, y):
    import numpy as np
    # Combine the feature vectors
    return np.concatenate((x, y), axis=1)

# Combine the TF-IDF matrix and FastText feature vectors
combined_features_vectors = combine_feature_vectors(tf_idf_matrix, word_embedding_matrix)


# Create a list of tuples for feature vector-author mapping
features_vectors_matrix = list(zip(combined_features_vectors, authors_list))

print(len(features_vectors_matrix[0][0]))


# Apply weight factor to the TF-IDF & FastText feature vectors
weight_factor_tfidf = 0.4  # Weight factor for TF-IDF
weight_factor_fasttext = 0.6  # Weight factor for FastText

# Create sample weights
sample_weights = [weight_factor_tfidf] * len(tf_idf_feature_names)
sample_weights += [weight_factor_fasttext] * len(word_embedding_feature_names) 

# Print the sample weights
print("Sample Weights:", sample_weights)
print("Number of Sample Weights:", len(sample_weights))

Feature Names: ['ؤثر', 'ؤجج', 'ؤجل', 'ؤخر', 'ؤده', 'ؤرق', 'ؤسء', 'ؤسس', 'ؤسف', 'ؤشر', 'ؤقت', 'ؤكد', 'ؤكل', 'ؤلف', 'ؤلم', 'ؤمر', 'ؤمن', 'ؤنث', 'ؤنس', 'ؤهل', 'ؤون', 'ؤيد', 'ئات', 'ئتم', 'ئلف', 'ئما', 'ئوه', 'ائت', 'ائتلافيه', 'ائتمانيه', 'ائم', 'ائه', 'االتاريخ', 'اب', 'ابا', 'ابارتهايد', 'ابب', 'ابت', 'ابتسامته', 'ابح', 'ابد', 'ابر', 'ابريلنيس', 'ابض', 'ابع', 'ابق', 'ابك', 'ابل', 'ابن', 'ابه', 'ابو', 'ابي', 'ات', 'اتا', 'اتالانتا', 'اتت', 'اتج', 'اتح', 'اتر', 'اتف', 'اتق', 'اتل', 'اتم', 'اتن', 'اتو', 'اتي', 'اثب', 'اثخ', 'اثر', 'اثل', 'اثم', 'اثن', 'اثنياتالملل', 'اثو', 'اجئ', 'اجب', 'اجتماعات', 'اجتماعيه', 'اجتماعيهالقاعده', 'اجتهاديه', 'اجج', 'اجد', 'اجدادكالمتجذره', 'اجر', 'اجل', 'اجه', 'احا', 'احب', 'احت', 'احتجاجيه', 'احتكاريه', 'احتياجات', 'احتياطيه', 'احج', 'احد', 'احذ', 'احز', 'احس', 'احص', 'احض', 'احط', 'احق', 'احك', 'احل', 'احم', 'احه', 'احي', 'اخا', 'اخت', 'اختباراته', 'اختصاصيه', 'اختلافات', 'اختياراته', 'اخد', 'اخذ', 'اخر', 'اخراللاجئ', 'اخرالمجتمع', 'اخرالموت', 'اخس', 'اخش

Processing by using SVM Model

In [94]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features_vectors, authors_list, test_size=0.2, random_state=42)


# Create and train the SVM model with the RBF kernel
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train, sample_weight=sample_weights)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: sample_weight and X have incompatible shapes: (6276,) vs (1275, 6276)
Note: Sparse matrices cannot be indexed w/boolean masks (use `indices=True` in CV).